In [5]:
# Cel 1: imports
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown
%matplotlib inline

from scipy.interpolate import pchip_interpolate
from genetic_algorithm_pfm.genetic_algorithm_pfm.algorithm import GeneticAlgorithm

# ⚙️ Explore the Decision Model ⚙️

This interactive model allows you to experiment with the decision process in a dynamic way. Adjust the sliders below to observe how changes in stakeholder importance (weights) influence the outcome of the **MINMAX optimization method**.
Consider the question: What happens to the design choice if one stakeholder’s preference becomes dominant? 


*Note: For our own analysis, this model was also executed using the **Preferendus method**, unfortunately the preferendus method requires access to internet, which is restricted by this web extension. (to make it work you should run it locally, but for demonstration purposes we think this was the easiest for the reader to setup)*

In [6]:
def objective_p1(X):
    """
    Minimised costs: O1 - Government
    """
    L = X[:, 0]
    B = X[:, 1] 
    H = X[:, 2]
    N = X[:, 3]
    c1 = 20000
    c2 = 13000
    c3 = 560000

    F1 = c1 * (2*L * (2 * H + B)) + c2 * 0.4 *  H * B + c3 * N
    p_value = pchip_interpolate([3e8,1.8e9 ,3e9 ], [100,65,0], F1)
    return p_value 


def objective_p2(X):
    """
    Maximised capacity: O2 - Vessels (aantal schepen per dag)
    """
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]

    c4 = 0.8
    Aship = 9691.5
    c5 = 0.4
    c6 =  2.3148e-4
    vvaar = 168000
    F2 = (L*B*c4)/(Aship * ((L/(vvaar*c5)+N*c6)))
    p_value = pchip_interpolate([100,300,500], [0, 20, 100], F2)
    return p_value

def objective_p3(X):
    """
    Maximum water elevation retaining height: O3 - Safety experts
    """
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]

    c7 = 1
    F3 = c7 * H
    p_value = pchip_interpolate([0, 10, 18, 20], [0, 20, 95, 100], F3)
    return p_value

def objective_p4(X):
    """
    Minimized environmental effects: O4 - Local residents
    """
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]

    c8 = 35
    c9 = 210
    c10 = 105

    F4 =  c8 * (2*L * (2 * H + B))  + c9 * 0.4 * H * B+ c10 * N
    p_value = pchip_interpolate([6e5,3e6,6e6], [100,50,0], F4)
    return p_value

def objective_p5(X):
    """
    Maximized car transport over lock - Municipality 
    """
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]

    c11 = 100

    F5 =  c11 * N
    p_value = pchip_interpolate([0,400,800], [0,80,100], F5)
    return p_value

def objective(X):
    """
    Objective function that is fed to the GA. Calles the separate preference functions that are declared above.

    :param variables: array with design variable values per member of the population. Can be split by using array
    slicing
    :return: 1D-array with aggregated preference scores for the members of the population.
    """
    # extract 1D design variable arrays from full 'variables' array

    # calculate the preference scores
    p_1 = objective_p1(X)
    p_2 = objective_p2(X)
    p_3 = objective_p3(X)
    p_4 = objective_p4(X)
    p_5 = objective_p5(X)

    w1 = 0.2
    w2 = 0.2
    w3 = 0.2
    w4 = 0.2
    w5 = 0.2
    
    return [w1, w2, w3, w4, w5], [p_1, p_2, p_3, p_4, p_5]

b1 = [300, 550]  # L
b2 = [30, 90]  # B
b3 = [0,20] # H
b4 = [0,8] # N

bounds = [b1, b2, b3, b4]

def constraint_1(X):
    """Maximum costs: 1,7 miljard"""
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]
    c1 = 20000
    c2 = 13000
    c3 = 560000
    
    obj_value = c1 * (2 * L * (2 * H + B)) + c2 * 0.4 * H * B + c3 * N

    return  obj_value - 2.7e9  # < 0

def constraint_2(X):
    """Minimum number of vessels/day: 60"""
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]
    c4 = 0.8
    Aship = 9691.5
    c5 = 0.4
    c6 = 2.3148e-4
    vvaar = 168000
    vessels_per_day = (L * B * c4) / (Aship * ((L / (vvaar * c5) + N * c6)))
    return -(vessels_per_day) + 60  # < 0

def constraint_3(X):
    """Minimum water capacity: 15.5 m"""
    H = X[:, 2]
    return -(H) + 15.5  # < 0

def constraint_4(X):
    """Maximum emissions: 5*10^6 ton CO2"""
    L = X[:, 0]
    B = X[:, 1]
    H = X[:, 2]
    N = X[:, 3]
    c8 = 35
    c9 = 210
    c10 = 105
    emissions = c8 * (2 * L * (2 * H + B)) + c9 * 0.4 * H * B + c10 * N
    return emissions - 5e6  # < 0

def constraint_5(X):
    """Minimum number of cars crossing/hour: 100"""
    N = X[:, 3]
    cars_per_hour = 100 * N
    return -(cars_per_hour) + 100  # < 0


cons = [
    ['ineq', constraint_1],
    ['ineq', constraint_2], 
    ['ineq', constraint_3], 
    ['ineq', constraint_4], 
    ['ineq', constraint_5]
    ]


In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import pchip_interpolate

# === Sliders voor de 5 gewichten ===
w1 = widgets.FloatSlider(description='Government', min=0, max=1, step=0.05, value=0.2)
w2 = widgets.FloatSlider(description='Shipping', min=0, max=1, step=0.05, value=0.2)
w3 = widgets.FloatSlider(description='Safety', min=0, max=1, step=0.05, value=0.2)
w4 = widgets.FloatSlider(description='Residents', min=0, max=1, step=0.05, value=0.2)
w5 = widgets.FloatSlider(description='Municipality', min=0, max=1, step=0.05, value=0.2)

run_button = widgets.Button(description='Run model', button_style='success')
out = widgets.Output()

def on_run_clicked(b):
    with out:
        clear_output(wait=True)
        print("Running both Genetic Algorithm paradigms (minmax & tetra)... ⏳")

        # Normaliseer gewichten
        weights = np.array([w1.value, w2.value, w3.value, w4.value, w5.value])
        weights = weights / np.sum(weights)

        def objective_weighted(X):
            p_values = [
                objective_p1(X),
                objective_p2(X),
                objective_p3(X),
                objective_p4(X),
                objective_p5(X),
            ]
            return weights, p_values

        paradigms = ['minmax']
        colours = ['orange', 'green']
        markers = ['o', '*']
        results = {}

        # Plot setup
        fig, axes = plt.subplots(3, 2, figsize=(15, 10))
        axes = axes.flatten()
        titles = ['Government', 'Shipping sector', 'Safety experts', 'Local residents', 'Municipality']
        x_labels = ['Costs [euro]', 'Capacity [ships/day]', 'Retaining height [m]', 'Environmental cost [euro]', 'Car transport [cars/day]']
        pref_curves = [
            ([3e8, 1.8e9, 3e9], [100, 65, 0]),
            ([100, 300, 500], [0, 20, 100]),
            ([0, 10, 18, 20], [0, 20, 95, 100]),
            ([6e5, 3e6, 6e6], [100, 50, 0]),
            ([0, 400, 800], [0, 80, 100]),
        ]

        for agg, color, marker in zip(paradigms, colours, markers):
            print(f"\n▶️ Running GA with aggregation = '{agg}'")
            options = {
                'n_bits': 8,
                'n_iter': 400,
                'n_pop': 500,
                'r_cross': 0.8,
                'max_stall': 8,
                'aggregation': agg,
                'var_type': 'real'
            }

            ga = GeneticAlgorithm(
                objective=objective_weighted,
                bounds=bounds,
                constraints=cons,
                options=options
            )

            best_score, best_solution, _ = ga.run(verbose=True)
            L, B, H, N = best_solution
            results[agg] = (best_score, best_solution)

            # Bereken F-waarden & preference scores
            X_best = np.array([best_solution])
            F_values = [
                20000 * (2 * L * (2 * H + B)) + 13000 * 0.4 * H * B + 560000 * N,
                (L * B * 0.8) / (9691.5 * ((L / (168000 * 0.4) + N * 2.3148e-4))),
                H,
                35 * (2 * L * (2 * H + B)) + 210 * 0.4 * H * B + 105 * N,
                100 * N
            ]
            pref_scores = [
                objective_p1(X_best)[0],
                objective_p2(X_best)[0],
                objective_p3(X_best)[0],
                objective_p4(X_best)[0],
                objective_p5(X_best)[0]
            ]

            # Plot voorkeurscurves + punten
            for i in range(5):
                x_curve, y_curve = pref_curves[i]
                x_plot = np.linspace(x_curve[0], x_curve[-1], 200)
                y_plot = pchip_interpolate(x_curve, y_curve, x_plot)
                if agg == 'minmax':
                    axes[i].plot(x_plot, y_plot, color='black', label='Preference curve')
                axes[i].scatter(F_values[i], pref_scores[i], color=color, s=100,
                                marker=marker, label=f'{agg} (L={L:.1f}, B={B:.1f}, H={H:.1f}, N={N:.1f})')
                axes[i].set_title(titles[i])
                axes[i].set_xlabel(x_labels[i])
                axes[i].set_ylabel('Preference score')
                axes[i].set_ylim(0, 102)
                axes[i].grid(linestyle='--')
                axes[i].legend()

        axes[5].axis('off')
        plt.tight_layout()
        plt.show()

        # Samenvatting
        print("\n📊 Results summary:")
        for agg in paradigms:
            best_score, sol = results[agg]
            L, B, H, N = sol
            print(f"  {agg.upper():7s} → Score: {best_score:.2f} | L={L:.2f}, B={B:.2f}, H={H:.2f}, N={N:.2f}")
        print(f"\nWeights used: {weights}")

run_button.on_click(on_run_clicked)
display(w1, w2, w3, w4, w5, run_button, out)

FloatSlider(value=0.2, description='Government', max=1.0, step=0.05)

FloatSlider(value=0.2, description='Shipping', max=1.0, step=0.05)

FloatSlider(value=0.2, description='Safety', max=1.0, step=0.05)

FloatSlider(value=0.2, description='Residents', max=1.0, step=0.05)

FloatSlider(value=0.2, description='Municipality', max=1.0, step=0.05)

Button(button_style='success', description='Run model', style=ButtonStyle())

Output()